In [ ]:
import os

import polars as pl
import plotly.express as px
from databricks import sql
from databricks.sdk.core import ApiClient
from databricks.sdk.service.iam import CurrentUserAPI
from dotenv import load_dotenv
from loguru import logger
from posit.connect.external.databricks import (
    ConnectStrategy,
    databricks_config,
    sql_credentials,
)
from posit.workbench.external.databricks import WorkbenchStrategy
from rich import inspect

load_dotenv()


## Configuration

Create the Databricks configuration.

In [ ]:
if os.environ.get("WORKBENCH_WEB_BASE_URL"):
    logger.info("Running in Posit Workbench")
    cfg = databricks_config(
        posit_workbench_strategy=WorkbenchStrategy(),
    )
    
elif os.environ.get("POSIT_PRODUCT") == "CONNECT":
    logger.info("Running in Posit Connect")
    cfg = databricks_config(
        posit_connect_strategy=ConnectStrategy(),
        host = "adb-3256282566390055.15.azuredatabricks.net"
    )

else:
    raise ValueError("Not running in Posit Workbench or Posit Connect.")


inspect(cfg)

In [ ]:
databricks_user = CurrentUserAPI(ApiClient(cfg)).me()
inspect(databricks_user)

## SQL Warehouse

Establish a connection to Databricks

In [ ]:
con = sql.connect(
    server_hostname=cfg.host,
    http_path="/sql/1.0/warehouses/e985c33f1db7502f",
    credentials_provider=sql_credentials(cfg),
)

inspect(con)

In [ ]:
sql_query = """
SELECT *
FROM `sol_eng_demo_nickp`.`default`.`lending_club`
LIMIT 100;
"""

df = (
    pl
    .read_database(sql_query, con)
    .with_columns(
        pl.col("loan_amnt", "funded_amnt", "funded_amnt_inv", "revol_bal").cast(pl.Int64),
        pl.col("installment", "annual_inc", "out_prncp", "out_prncp_inv", "total_pymnt").cast(pl.Float64),
        pl.col("int_rate", "revol_util").str.replace("%", "").cast(pl.Float64) / 100
    )
)

df

In [ ]:
px.histogram(df, x="loan_amnt")